# ...

In [ ]:
# %cd /content
# ! rm -r competition

In [ ]:
! git clone https://github.com/Tikquuss/competition

In [ ]:
%cd competition/code

# Imports

In [ ]:
# Imports from our code
from utils import DATA_PATH, DIR_PATH_FIGURES, DIR_PATH_SUBMISSIONS, H, W
from utils import predict_nontest, predict_test, save_for_submission, eval
from data import  get_dataset
from plotter import plot_cdf, custom_imshow, confusion_matrix, scores, plot_confusion_matrix, show_example_images
from model_tree import RandomForestClassifier

from sklearn.ensemble import RandomForestClassifier as sklearn_RandomForestClassifier

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import tqdm
import itertools
from collections import Counter
import time
from pathlib import Path
import pickle

#import warnings
#warnings.filterwarnings('ignore')

# Training & Validation & Test dataset

**Normal training data**

In [ ]:
#HEIGHT, WIDTH = 60, 60
#HEIGHT, WIDTH = 25, 25
#HEIGHT, WIDTH = 10, 10
HEIGHT, WIDTH = 5, 5
#HEIGHT, WIDTH = None, None

scaler_class=None
#scaler_class='standard_scaler'
scaler_class='min_max_scaler'

train_pct, holdout_pct = 80, 10
#train_pct, holdout_pct = 90, 10
#train_pct, holdout_pct = 100,0

seed=0
n_classes=24

In [ ]:
IDs_test, (X_tr, Y_tr, X_ht_test, Y_ht_test, X_val, Y_val, X_all, Y_all, X_test, X_test_all, X_ht_test_all, d) = get_dataset(
    train_pct=train_pct, holdout_pct=holdout_pct, k_fold=False, HEIGHT=HEIGHT, WIDTH=WIDTH, do_over_sampling=False, do_under_sampling=False,
    scaler_class=scaler_class, is_pytorch=False, seed=seed
)

In [ ]:
print(X_all.min(), X_all.max())
_=show_example_images(X_all.reshape(-1, H if HEIGHT is None else HEIGHT, W if HEIGHT is None else WIDTH), Y_all, n_imgs=15, mono = 'viridis')

**Training data for k-fold cross validation**

In [ ]:
# IDs_test, kfold_iterator = get_dataset(
#     train_pct=train_pct, holdout_pct=holdout_pct, k_fold=True, HEIGHT=HEIGHT, WIDTH=WIDTH, do_over_sampling=False, do_under_sampling=False,
#     scaler_class=scaler_class, is_pytorch=False, seed=seed
# )

# My random forest

In [ ]:
# : python train_tree.py --model_name test --n_estimators 100 --max_depth 100 --max_samples 1.0 --max_features sqrt --sklearn False --SIZE 28 --train_pct 90 --holdout_pct 10 --seed 0

**OR**

In [ ]:
fileName="test_random_forest"

In [ ]:
forest = RandomForestClassifier(
                 n_estimators=10,
                 criterion='gini',
                 max_depth=100,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 max_features='sqrt',
                 max_leaf_nodes=None,
                 min_impurity_decrease=0.0,
                 bootstrap=True,
                 random_state=None,
                 verbose=0,
                 max_samples=0.1,
          )

In [ ]:
if train_pct + holdout_pct < 100 : # validation data
    forest.fit(X_tr, Y_tr)
    train_acc = eval(forest, X_tr, Y_tr)
    val_acc = eval(forest, X_val, Y_val)
    test_acc = predict_nontest(forest, X_ht_test, Y_ht_test, seed=seed)

    conf_matrix_1 = confusion_matrix(Y_tr, forest.predict(X_tr), n_classes=n_classes)
    conf_matrix_2  = confusion_matrix(Y_ht_test, forest.predict(X_ht_test), n_classes=n_classes)

    Y_hat_A, Y_hat_B = predict_test(forest, X_test)
else : # No validation data

    forest.fit(X_all, Y_all)
    train_acc = eval(forest, X_all, Y_all)
    val_acc = -1
    test_acc = predict_nontest(forest, X_ht_test_all, Y_ht_test, seed=seed)

    conf_matrix_1 = confusion_matrix(Y_all, forest.predict(X_all), n_classes=n_classes)
    conf_matrix_2 = confusion_matrix(Y_ht_test, forest.predict(X_ht_test_all), n_classes=n_classes)

    Y_hat_A, Y_hat_B = predict_test(forest, X_test_all)

node_count = forest.node_count()

print("train acc : {:7.4f} % \nvalidation acc : {:7.4f} % \nnumber of nodes  : {:7}".format(train_acc * 100, val_acc * 100, node_count))
print("test : ", test_acc)


In [ ]:
plot_confusion_matrix(conf_matrix_1, fileName=f"{fileName}_train")
_ = scores(conf_matrix_1, fileName=f"{fileName}_train")

plot_confusion_matrix(conf_matrix_2, fileName=f"{fileName}_test")
_ = scores(conf_matrix_2, fileName=f"{fileName}_test")

In [ ]:
save_for_submission(IDs_test, Y_hat_A, Y_hat_B, fileName=f"{fileName}.csv")
i=0
_=show_example_images(X_test[i].reshape(-1, H if HEIGHT is None else HEIGHT, W if HEIGHT is None else WIDTH), [Y_hat_A, Y_hat_B][i], n_imgs=15, mono = 'viridis')

In [ ]:
with open(f"{DIR_PATH_FIGURES}/{fileName}.pickle" ,"wb") as file_handle:
    to_save = {
      "model" : forest,
      "perfs" : [train_acc, val_acc, test_acc],
      "others" : [conf_matrix_1, conf_matrix_2],
      "Y_hat_test" : [Y_hat_A, Y_hat_B]
    }
    pickle.dump(to_save, file_handle, pickle.HIGHEST_PROTOCOL)

# New Data

In [ ]:
HEIGHT, WIDTH = None, None

scaler_class=None
#scaler_class='standard_scaler'
scaler_class='min_max_scaler'

train_pct, holdout_pct = 80, 10
#train_pct, holdout_pct = 90, 10
#train_pct, holdout_pct = 100,0

seed=0
n_classes=24

IDs_test, (X_tr, Y_tr, X_ht_test, Y_ht_test, X_val, Y_val, X_all, Y_all, X_test, X_test_all, X_ht_test_all, d) = get_dataset(
    train_pct=train_pct, holdout_pct=holdout_pct, k_fold=False, HEIGHT=HEIGHT, WIDTH=WIDTH, do_over_sampling=False, do_under_sampling=False,
    scaler_class=scaler_class, is_pytorch=False, seed=seed
)

# Sklearn random forest

In [ ]:
fileName="test_random_forest_sklearn"

In [ ]:
forest = sklearn_RandomForestClassifier(
                 n_estimators=10,#0, #, # 128
                 criterion='gini',
                 max_depth=100, #None, # 8
                 min_samples_split=2,
                 min_samples_leaf=1,
                 max_features='sqrt',
                 max_leaf_nodes=None,
                 min_impurity_decrease=0.0,
                 bootstrap=True,
                 random_state=0,
                 verbose=0,
                 max_samples=None)

In [ ]:
if train_pct + holdout_pct < 100 : # validation data
    forest.fit(X_tr, Y_tr)
    train_acc = eval(forest, X_tr, Y_tr)
    val_acc = eval(forest, X_val, Y_val)
    test_acc = predict_nontest(forest, X_ht_test, Y_ht_test, seed=seed)

    conf_matrix_1 = confusion_matrix(Y_tr, forest.predict(X_tr), n_classes=n_classes)
    conf_matrix_2  = confusion_matrix(Y_ht_test, forest.predict(X_ht_test), n_classes=n_classes)

    Y_hat_A, Y_hat_B = predict_test(forest, X_test)
else : # No validation data

    forest.fit(X_all, Y_all)
    train_acc = eval(forest, X_all, Y_all)
    val_acc = -1
    test_acc = predict_nontest(forest, X_ht_test_all, Y_ht_test, seed=seed)

    conf_matrix_1 = confusion_matrix(Y_all, forest.predict(X_all), n_classes=n_classes)
    conf_matrix_2 = confusion_matrix(Y_ht_test, forest.predict(X_ht_test_all), n_classes=n_classes)

    Y_hat_A, Y_hat_B = predict_test(forest, X_test_all)

node_count = 0

print("train acc : {:7.4f} % \nvalidation acc : {:7.4f} % \nnumber of nodes  : {:7}".format(train_acc * 100, val_acc * 100, node_count))
print("test : ", test_acc)

In [ ]:
# plot_confusion_matrix(conf_matrix_1, fileName=f"{fileName}_train")
# _ = scores(conf_matrix_1, fileName=f"{fileName}_train")

# plot_confusion_matrix(conf_matrix_2, fileName=f"{fileName}_test")
# _ = scores(conf_matrix_2, fileName=f"{fileName}_test")

In [ ]:
# save_for_submission(IDs_test, Y_hat_A, Y_hat_B, fileName=f"{fileName}.csv")
# i=0
#_ = show_example_images(X_test[i].reshape(-1, H if HEIGHT is None else HEIGHT, W if HEIGHT is None else WIDTH), [Y_hat_A, Y_hat_B][i], n_imgs=15, mono = 'viridis')

In [ ]:
# with open(f"{DIR_PATH_FIGURES}/{fileName}.pickle" ,"wb") as file_handle:
#     to_save = {
#       "model" : forest,
#       "perfs" : [train_acc, val_acc, test_acc],
#       "others" : [conf_matrix_1, conf_matrix_2],
#       "Y_hat_test" : [Y_hat_A, Y_hat_B]
#     }
#     pickle.dump(to_save, file_handle, pickle.HIGHEST_PROTOCOL)

# Hyperameter seachs Sklearn Random Forest

In [ ]:
fileName="test_random_forest_sklearn_hpsearch"
log_dir = Path(DIR_PATH_FIGURES).parent.absolute()

log_dir = os.path.join(log_dir, f"{fileName}")
DIR_PATH_FIGURES__ = os.path.join(log_dir, "figures")
DIR_PATH_SUBMISSIONS__ = os.path.join(log_dir, "submissions")
os.makedirs(DIR_PATH_SUBMISSIONS__, exist_ok=True)
os.makedirs(DIR_PATH_FIGURES__, exist_ok=True)

In [ ]:
""""
m : the number of trees
r : maximum depth
n_prime : number of samples to draw to train each base tree
d_prime : number of features to consider for each tree  ($d' = \lfloor \sqrt{d} \rfloor$ ...)
f  : purity criteria (Gini index or entropy).
"""

all_m = np.array([1,  10])
all_r = np.array([10,  50])
all_n_prime = [0.1, 1.0]
all_d_prime = [28, 1.0] # 28 = sqrt(d)
#all_f = ["gini", "entropy"]
all_f = ["gini"]

# all_m = np.array([1,  50,  100, 200, 300])
# all_r = np.array([10,  100, 150, 200, 300])
# all_n_prime = [0.1, 0.5,  1.0]
# all_d_prime = [28, 0.5, 1.0] # 28 = sqrt(d)
# #all_f = ["gini", "entropy"]
# all_f = ["gini"]

all_hparams = itertools.product(all_m, all_r, all_n_prime, all_d_prime, all_f)
all_hparams = list(all_hparams)
len(all_hparams)

In [ ]:
all_performances = {}
all_models = {}

for m, r, n_prime, d_prime, f in tqdm.tqdm(all_hparams, desc="Hyp search") :

    train_start = time.time()

    forest = sklearn_RandomForestClassifier(
                    n_estimators=m, # m
                    criterion=f, # f
                    max_depth=r, # r
                    min_samples_split=2,
                    min_samples_leaf=1,
                    max_features=d_prime, # d'
                    max_leaf_nodes=None,
                    min_impurity_decrease=0.0,
                    bootstrap=True,
                    random_state=0,
                    verbose=0,
                    max_samples=n_prime # n'
                    )


    forest.fit(X_tr, Y_tr)
    train_acc = eval(forest, X_tr, Y_tr)
    val_acc = eval(forest, X_val, Y_val)

    training_time = time.time() - train_start

    # m, r, n_prime, d_prime, f
    key = f"m={m}_r={r}_n'={n_prime}_d'={d_prime}_f={f}"
    print("\n", key, train_acc * 100, val_acc * 100, training_time)

    all_models[key] = forest
    all_performances[key] = [train_acc, val_acc]

In [ ]:
with open(f"{log_dir}/all_performances_sktree.pickle","wb") as file_handle:
    pickle.dump(all_performances, file_handle, pickle.HIGHEST_PROTOCOL)

with open(f"{log_dir}/all_models_sktree.pickle","wb") as file_handle:
    pickle.dump(all_models, file_handle, pickle.HIGHEST_PROTOCOL)

In [ ]:
rows, cols = 1, 1
figsize = (6, 4)
figsize=(cols*figsize[0], rows*figsize[1])
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(rows, cols, 1)

values = np.array(list(all_performances.values()))
values = [values[:,0], values[:,1]]
for v, model_name in zip(values, ['training', 'validation']) :
    ax.hist(x=v,
        label=model_name, color=None,
        histtype='step',
        align='mid',
        orientation='vertical',
        log=False,
        stacked=False,
        density=True,
        rwidth=None,
        #rwidth=0.8,
    )

ax.grid()
ax.legend(prop={'size': 15})

fileName=f"histogramm_perf"
plt.savefig(f"{DIR_PATH_FIGURES__}/{fileName}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')
#print(fileName)

In [ ]:
x = np.array(list(all_performances.values()))[:,0]
_ = plot_cdf(x, fileName="hp_search_sktree_cdf_train", dpf=DIR_PATH_FIGURES__)
#plt.close()
x = np.array(list(all_performances.values()))[:,1]
_ = plot_cdf(x, fileName="hp_search_sktree_cdf_val", dpf=DIR_PATH_FIGURES__)

In [ ]:
all_n_prime_d_prime = list(itertools.product(all_n_prime, all_d_prime))
all_n_prime_d_prime

In [ ]:
n_imgs=len(all_n_prime_d_prime)

L=3
C=n_imgs//L + 1*(n_imgs%L!=0)

C=3
L=n_imgs//C + 1*(n_imgs%C!=0)

figsize=(C*15, L*10)
#figsize=(C*6, L*4)
fig = plt.figure(figsize=figsize)

train_or_val = 0 # train
train_or_val = 1 # vam
values =  np.array(list(all_performances.values()))[:,train_or_val]
factor = 100
minmin = factor*min(values)
maxmax = factor*max(values)

axlist = []
iii = 0
for n_prime, d_prime in all_n_prime_d_prime[:L*C] :
    ax = fig.add_subplot(L, C, iii+1)

    # gamma : all_m
    # lrs : all_r
    img_data = np.empty((len(all_m), len(all_r)))
    for i, m in enumerate(all_m) :
        for j, r in enumerate(all_r) :
            f = "gini"
            img_data[i][j] = all_performances[f"m={m}_r={r}_n'={n_prime}_d'={d_prime}_f={f}"][train_or_val]

    img_data = img_data*factor

    #print(img_data)
    imshow_kwarg = {'vmin':minmin, 'vmax':maxmax,
                    #"extent":(-5,5,-5,5),
                    "aspect":'auto'}
    img = custom_imshow(img_data,
                  ax=ax, fig=fig,
                  add_text=True, n_decimals=2,
                  yticklabels=all_m,
                  xticklabels=all_r,
                  y_label="number of trees", x_label="maximum depth",
                  rotation_x=90, rotation_y=0,
                  imshow_kwarg = imshow_kwarg,
                  colorbar = False,
                  show=False,
                  #fileName=f"m_vs_r_hpsearch_sklearn"
                  )

    ax.set_title(f"(n', d')=({n_prime}, {d_prime})", fontsize=30, loc='right', y=-0.1)#, y=0.5, pad=-15, )

    axlist.append(ax)
    iii+=1

fig.colorbar(img, ax=axlist, fraction=0.046, pad=0.04, aspect=60, location="top")

fileName=f"m_vs_r_hpsearch_sklearn_{'train' if train_or_val==0 else 'val'}"
plt.savefig(f"{DIR_PATH_FIGURES__}/{fileName}"  + '.pdf', dpi=300, bbox_inches='tight', format='pdf')

plt.show()

In [ ]:
kv = {k:v[1] for k, v in all_performances.items()} # val
hparam_best = max(kv, key=all_performances.get)
hparam_best, all_performances[hparam_best]

In [ ]:
from sklearn.ensemble import RandomForestClassifier as sklearn_RandomForestClassifier

In [ ]:
m=200
r=100
n_prime=1.0
d_prime=28
f="gini"
forest = sklearn_RandomForestClassifier(
                n_estimators=m, # m
                criterion=f, # f
                max_depth=r, # r
                min_samples_split=2,
                min_samples_leaf=1,
                max_features=d_prime, # d'
                max_leaf_nodes=None,
                min_impurity_decrease=0.0,
                bootstrap=True,
                random_state=0,
                verbose=0,
                max_samples=n_prime # n'
                )


forest.fit(X_all, Y_all)
train_acc = eval(forest, X_all, Y_all)
val_acc = -1
test_acc = predict_nontest(forest, X_ht_test_all, Y_ht_test, seed=seed)

conf_matrix_1 = confusion_matrix(Y_all, forest.predict(X_all), n_classes=n_classes)
conf_matrix_2 = confusion_matrix(Y_ht_test, forest.predict(X_ht_test_all), n_classes=n_classes)

Y_hat_A, Y_hat_B = predict_test(forest, X_test_all)

print(train_acc * 100, val_acc*100, test_acc)

In [ ]:
plot_confusion_matrix(conf_matrix_1, fileName=f"{fileName}_train", dpf=DIR_PATH_FIGURES__)
_ = scores(conf_matrix_1, fileName=f"{fileName}_train", dpf=DIR_PATH_FIGURES__)

plot_confusion_matrix(conf_matrix_2, fileName=f"{fileName}_test", dpf=DIR_PATH_FIGURES__)
_ = scores(conf_matrix_2, fileName=f"{fileName}_test", dpf=DIR_PATH_FIGURES__)

In [ ]:
save_for_submission(IDs_test, Y_hat_A, Y_hat_B, fileName=f"{fileName}.csv", dps=DIR_PATH_SUBMISSIONS__)
i=0
_=show_example_images(X_test[i].reshape(-1, H if HEIGHT is None else HEIGHT, W if HEIGHT is None else WIDTH), [Y_hat_A, Y_hat_B][i], n_imgs=15, mono = 'viridis')

In [ ]:
with open(f"{DIR_PATH_FIGURES__}/{fileName}.pickle" ,"wb") as file_handle:
    to_save = {
      "model" : forest,
      "perfs" : [train_acc, val_acc, test_acc],
      "others" : [conf_matrix_1, conf_matrix_2],
      "Y_hat_test" : [Y_hat_A, Y_hat_B]
    }
    pickle.dump(to_save, file_handle, pickle.HIGHEST_PROTOCOL)

# Other Sklearn methods

In [ ]:
# ! pip install xgboost

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
import xgboost

from sklearn.ensemble import  ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier as sklearn_RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

In [ ]:
def train_and_plot_sklean(sk_model) :
    """forest, Y_hat_test, strain, s_ht_test = train_and_plot_sklean(sk_model)"""
    clf = sk_model.fit(X_all, Y_all)
    strain, s_ht_test = clf.score(X_all, Y_all), clf.score(X_ht_test_all, Y_ht_test)
    print(strain, s_ht_test)
    Y_hat_test = clf.predict(X_ht_test_all)
    #print(Counter(Y_hat_test))
    return clf, Y_hat_test, strain, s_ht_test

In [ ]:
# https://www.kaggle.com/code/davidfumo/comparing-11-classification-models
classifiers = [
    SVC(kernel="rbf", C=0.025, probability=True),
    #SVC(gamma='auto', probability=True), # 2
    LinearSVC(),
    #LogisticRegression(random_state=0), # 4
    #KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    XGBClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(), # 1
    GaussianNB(), # 5
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    MLPClassifier(
        hidden_layer_sizes=(d, 500, n_classes), activation='relu', solver='adam', alpha=0.0001,
        batch_size=X_tr.shape[0],
        learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=100,
        shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9,
        nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999,
        epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
    ]

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Training set", "Holdout test set"]
log = pd.DataFrame(columns=log_cols)
all_clf = {}

In [ ]:
for sk_model in classifiers:
    clf, Y_hat_test, strain, s_ht_test = train_and_plot_sklean(sk_model)

    name = clf.__class__.__name__
    while name in all_clf.keys() : name += "_1"
    print(name)
    all_clf[name] = clf

    log_entry = pd.DataFrame([[name, strain, s_ht_test]], columns=log_cols)
    log = log.append(log_entry)

    print("="*30)

print("="*30)

In [ ]:
log_sort = log.sort_values(by=['Holdout test set'], ascending=False)
for index, row in log_sort.iterrows() :
    print(f"{row['Classifier']} & {round(row['Training set']*100, 2)} & {round(row['Holdout test set']*100, 2)} \\\ \hline")

In [ ]:
sns.set_color_codes("muted")
sns.barplot(x="Training set", y='Classifier', data=log_sort, color="b")
plt.xlabel("Accuracy")
plt.title('Training set')
plt.show()

sns.set_color_codes("muted")
sns.barplot(x="Holdout test set", y='Classifier', data=log_sort, color="b")
plt.xlabel("Accuracy")
plt.title('Holdout test set')
plt.show()

In [ ]:
all_Y = []
for k, clf in all_clf.items() :
    s_ht_test = clf.score(X_ht_test_all, Y_ht_test)
    print(k, s_ht_test)
    Y_hat_test = clf.predict(X_ht_test_all)
    all_Y.append(Y_hat_test)